In [ ]:
import numpy as np
from scipy.spatial import Delaunay
import cv2
import os
import math
from tqdm import tqdm
import random

In [ ]:
FFT = True # whether to use FFT-based down-sampling
scale = 10 # down-sampling scale
HR_dir="../Set5/" # origin image folder

In [ ]:
# FFT high-pass filtering
def high_pass_filtering(image, radius, n):
    fft = cv2.dft(np.float32(image), flags=cv2.DFT_COMPLEX_OUTPUT)
    dshift = np.fft.fftshift(fft)

    rows, cols = image.shape[:2]
    mid_row, mid_col = int(rows / 2), int(cols / 2)

    mask = np.zeros((rows, cols, 2), np.float32)
    for i in range(0, rows):
        for j in range(0, cols):
            d = math.sqrt(pow(i - mid_row, 2) + pow(j - mid_col, 2))
            try:
                mask[i, j, 0] = mask[i, j, 1] = 1 / (1 + pow(radius / d, 2*n))
            except ZeroDivisionError:
                mask[i, j, 0] = mask[i, j, 1] = 0
    fft_filtering = dshift * mask
    ishift = np.fft.ifftshift(fft_filtering)
    image_filtering = cv2.idft(ishift)
    image_filtering = cv2.magnitude(
        image_filtering[:, :, 0], image_filtering[:, :, 1])
    cv2.normalize(image_filtering, image_filtering, 0, 1, cv2.NORM_MINMAX)
    return image_filtering

In [ ]:
class Triangle:
    def __init__(self, coordinate, color) -> None:
        self.coordinate = coordinate
        self.color = color
        self.A, self.B, self.C = self.coordinate[0], self.coordinate[1], self.coordinate[2]
        self.ABx = self.B[0] - self.A[0]
        self.ACx = self.C[0] - self.A[0]
        self.ABy = self.B[1] - self.A[1]
        self.ACy = self.C[1] - self.A[1]

    def barycentric(self, P):
        # compute the barycentric coordinates of P
        s1 = np.array([[self.ABx, self.ACx, self.A[0] - P[i][0]]
                       for i in range(P.shape[0])])
        s2 = np.array([[self.ABy, self.ACy, self.A[1] - P[i][1]]
                       for i in range(P.shape[0])])
        _u = np.cross(s1, s2)
        ans = [[1-(u[0]+u[1])/u[2], u[0]/u[2], u[1]/u[2]]
               if abs(u[2]) > 0 else [-1, 1, 1]
               for u in _u]
        return np.array(ans)


In [ ]:
corner = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
for root, dirs, files in os.walk(HR_dir):
    for file in tqdm(files, leave=True):
        HR_img = cv2.imread(os.path.join(root, file))
        point_num = HR_img.shape[0]*HR_img.shape[1]//scale
        if FFT:
            fft_HR_img = cv2.cvtColor(HR_img, cv2.COLOR_BGR2GRAY)
            fft_HR_img = high_pass_filtering(fft_HR_img, 30, 1)
            #! TODO This method may lead to prob*fft_HR_img[i][j] > 1
            prob = point_num/(np.sum(fft_HR_img))
            points = []
            for i in range(HR_img.shape[0]):
                for j in range(HR_img.shape[1]):
                    if random.random() <= prob*fft_HR_img[i][j]:
                        points.append([i, j])
        else:  # sample randomly
            points = np.random.random((point_num, 2))
            points = np.array(
                points*[HR_img.shape[0]-1, HR_img.shape[1]-1], dtype=int)
        HR_img_corner = np.array(
            corner*[HR_img.shape[0]-1, HR_img.shape[1]-1], dtype=int)
        points = np.concatenate((HR_img_corner, np.array(points)))

        delaunay = Delaunay(points)  # execute Delaunay triangualtion algorithm
        points_coord = points[delaunay.simplices]
        tris = [Triangle(c, np.array([HR_img[c[0][0]][c[0][1]], HR_img[c[1][0]]
                                      [c[1][1]], HR_img[c[2][0]][c[2][1]]])) for c in points_coord]

        computed = set()
        LR_img = np.zeros((HR_img.shape[0], HR_img.shape[1], 3))
        for t in tqdm(tris, leave=False):  # for each triangle
            bbox = np.array([[np.min(t.coordinate[:, 0]), np.min(t.coordinate[:, 1])],
                             [np.max(t.coordinate[:, 0]), np.max(t.coordinate[:, 1])]])
            coord = np.array([[x, y] for x in range(bbox[0][0], bbox[1][0]+1)
                              for y in range(bbox[0][1], bbox[1][1]+1)])
            u = t.barycentric(coord)
            colors = np.dot(u, t.color)  # interpolate color
            for i in range(coord.shape[0]):
                x, y = coord[i]
                if (x, y) in computed:  # prevent from repeated computation
                    continue
                barycentric = u[i]
                # in this triangle
                if barycentric[0] >= 0 and barycentric[1] >= 0 and barycentric[2] >= 0:
                    computed.add((x, y))
                    LR_img[x][y] = colors[i]
        # save LR image
        LR_img = LR_img.astype(np.uint8)
        output_dir = '../delaunay/{}/x{}'.format(
            'FFT' if FFT else 'random', scale)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        cv2.imwrite(os.path.join(output_dir, file), LR_img)
